In [1]:
import sys
sys.path.append('../code')

from settings import VECTOR_SIZE,  CLASSIFIER_EPOCHS


In [2]:
from gensim.models import Word2Vec
word_vectors = Word2Vec.load('walks/model').wv

In [3]:
from pathlib import Path
import pandas as pd
from models import ClassifierSimple
import torch
model = ClassifierSimple()    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if Path('rdf2vecClassfier.pth').is_file():
    print('found trained model! Loading :)')
    model.load_state_dict(torch.load('rdf2vecClassfier.pth'))
    history = pd.read_csv('log.csv')
    model = model.to(device)
else:
    print('model not found. Train it with ''train_rdf2vec_classifier.ipynb')

/home/otautz/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


found trained model! Loading :)


RuntimeError: Error(s) in loading state_dict for ClassifierSimple:
	size mismatch for layers.1.weight: copying a param with shape torch.Size([64, 300]) from checkpoint, the shape in current model is torch.Size([64, 384]).

In [ ]:
from rdflib import Graph, URIRef
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
from utils_graph import clean_graph, get_entities


g_train = Graph()
g_val = Graph()
g_test = Graph()

g_train = g_train.parse('FB15k-237/train_1000.nt', format='nt')
g_val   = g_val.parse('FB15k-237/valid_1000.nt', format='nt')
g_test  = g_test.parse('FB15k-237/test_1000.nt', format='nt')


# clean graphs 
# number of triples removed should be low, a few hundred
print(f"removed {clean_graph(g_train,word_vectors)} triples from training set")
print(f"removed {clean_graph(g_val,word_vectors)} triples from validation set")
print(f"removed {clean_graph(g_test,word_vectors)} triples from test set")

entities = get_entities((g_train,g_val,g_test))

In [ ]:
from utils_eval import eval_ranks, mean_rank, mean_reciprocal_rank, hitsAt

In [ ]:
# tested scoring on cpu
import gc
from utils.eval import eval_model
torch.set_num_threads(32)
   

stats, pt = eval_model(model,g_train,lambda x: word_vectors[x])

for k,v in stats.items():
    print(f"k:\tv"

In [ ]:
print('here!')